In [60]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt

In [61]:
def get_images(image_path):

    image = tf.image.decode_jpeg(tf.io.read_file(image_path))
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [512, 512])
    image = image[tf.newaxis, :]

    return image

In [62]:
content_image = get_images('./content.jpg')
style_image = get_images('./style.jpg')

In [63]:
vgg = VGG19(
    include_top=False,
    weights='imagenet',
)
vgg.trainable = False

In [64]:
def get_model(content_layers, style_layers):
    
    content = [vgg.get_layer(layer).output for layer in content_layers]
    style = [vgg.get_layer(layer).output for layer in style_layers]

    model = Model([vgg.input], [content, style])
    return model

In [65]:
content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

In [66]:
model = get_model(content_layers, style_layers)

In [67]:
req_style = model(preprocess_input(style_image * 255))[1]
req_content = model(preprocess_input(content_image * 255))[0]

In [68]:
def gram_matrix(output, normalize=True):
    
    matrix = tf.linalg.einsum('nija,nijb->nab', output, output)
    shape = tf.shape(matrix)
    N = tf.cast(shape[1] * shape[2], tf.float32)
    
    if normalize:
        matrix /= N

    return matrix

In [69]:
def calc_loss(output, ratio):

    content = output[0]
    style = output[1]

    style_loss = tf.add_n([tf.reduce_mean((gram_matrix(style[i]) - gram_matrix(req_style[i]))**2) for i in range(len(style_layers))]) / len(style_layers)
    content_loss = tf.add_n([tf.reduce_mean((content[i] - req_content[i])**2) for i in range(len(content_layers))]) / len(content_layers)

    return ratio * content_loss + (1-ratio) * style_loss

In [70]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.02)
def clip(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

In [71]:
def train(image, ratio):
    with tf.GradientTape() as tape:
        img = preprocess_input(image*255)
        output = model(img)
        loss = calc_loss(output, ratio)

    grad = tape.gradient(loss, image)
    optimizer.apply_gradients([(grad, image)])
    image.assign(clip(image))

In [72]:
def make_image(epochs, ratio):
    image = tf.Variable(content_image)
    for _ in range(epochs):
        train(image, ratio)
    
    plt.imshow(image[0])

In [ ]:
make_image(1000, 0.99999)